# Data Preprocessing and Exploratory Data Analysis

In this notebook, i detail my thought process as i work through the data preprocessing while also drawing insights from the visualisations during exploratory data analysis.

The preprocessing and EDA will be performed on the training data, and the respective preprocessing / transformations will be made to the testing data set at the end.

Below is a table of contents to aid navigation to specific sections of the notebook.

__Note__: Majority of the functions used in this notebook are called from the `cleaning.py` and `visualisation.py` scripts in the `utils` folder.

## Table of Contents <a id='table_contents'></a>

#### [1. Basic Data Preprocessing](#preprocessing)
* [Initial Analysis of our Data](#initial_analysis)
* [Correcting Data Types](#data_types)
* [Handling Missing Data](#missing_data)

### Importing Libraries

In [3]:
# Standard Tools
from scipy import stats
import pandas as pd
import numpy as np
import pickle

# Visualisation Tools
import matplotlib.pyplot as plt
import seaborn as sns

# Custom Functions
from utils import visualisation as viz
from utils import cleaning

# Misc
import warnings
warnings.filterwarnings('ignore')

# Pandas Options
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

# Plotting Options
plt.rcParams['axes.facecolor'] = 'white'
plt.style.use('fivethirtyeight')

# Magic Functions
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

## Basic Data Preprocessing <a id='preprocessing'></a>

In this section, we want to take a preliminary look at our data. This allows us to __identify potential errors__ within our data such as __missing values, incorrect data entry / data types and skewed features__, in order to rectify them.  

### Loading Data and Initial Analysis <a id='initial_analysis'></a>

In [4]:
train = pd.read_csv('./data/train.csv')

In [5]:
train.head()

,id,industry,state,request_date,term,employee_count,business_new,business_type,location,other_loans,loan_amount,insured_amount,default_status
0,4050975007,Others,VA,27-Apr-10,34,4,New,0,Rural,N,"$35,000.00","$35,000.00",1
1,3735095001,Manufacturing,CA,05-Nov-09,107,1,New,0,Rural,N,"$15,000.00","$13,500.00",1
2,3936555004,Trading,CA,26-Feb-10,84,1,New,0,Rural,Y,"$265,000.00","$100,000.00",0
3,4130405000,Engineering,MI,10-Jun-10,240,21,New,0,Rural,N,"$255,000.00","$255,000.00",0
4,4263615008,Education,NH,23-Sep-10,36,1,Existing,0,Rural,N,"$13,300.00","$6,650.00",0


In [12]:
# summary report that does basic cleaning and identifies common issues such as missing data or skewed features
train = cleaning.data_report(train, reminders=True)

SUMMARY REPORT FOR DATASET

Cleaning dataframe columns ...
Removing whitespaces ...
Replacing dashes with underscores ...
Changing to lower case ...
Dataframe column headers have been formatted.

The data has 2402 observations and 13 features.
Numerical Features : 5
Categorical Features : 8

The data has missing values.
- To save the features and their respective missing values, call the variable_missing_percentage function.

Methods to handle missing values for Numerical Features:
1. Impute missing values with Mean / Median / KNN Imputer
2. Drop feature with high proportion of missing values.
3. For time series data, consider Linear Interpolation / Back-filling / Forward-filling

Methods to handle missing values for Categorical Features:
1. Impute missing values with Mode / KNN Imputer
2. Classify all the missing values as a new category.
3. Drop feature with high proportion of missing values.

There are Numerical Features within the data that have skewness greater than 1 in magnitude

In [16]:
# dropping the 'id' feature as we can use the Dataframe's index
train = train.drop('id', axis=1)

### Checking Feature Data Types <a id='data_types'></a>

From the above summary report on our training dataset, we see that we have a few issues such as missing data and skewed variables. 

But before we handle those issues, we need to check if the features have the correct data types. 

In [19]:
# checking data types
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2402 entries, 0 to 2401
Data columns (total 12 columns):
industry          2401 non-null object
state             2402 non-null object
request_date      2402 non-null object
term              2402 non-null int64
employee_count    2402 non-null int64
business_new      2402 non-null object
business_type     2402 non-null int64
location          2402 non-null object
other_loans       2402 non-null object
loan_amount       2402 non-null object
insured_amount    2402 non-null object
default_status    2402 non-null int64
dtypes: int64(4), object(8)
memory usage: 150.2+ KB


There two features `loan_amount` and `insured_amount`  have __incorrect data types__, as they have 'objects' data type when they are in fact numerical data.

From our initial look at the data, we  know that the __problem is due to the '$' character__ being included in the values.

In [23]:
# removing '$' and correcting data types
train['loan_amount'] = train['loan_amount'].replace('[\$,]', '', regex=True).astype(float)
train['insured_amount'] = train['insured_amount'].replace('[\$,]', '', regex=True).astype(float)

Furthermore, we will convert the `request_date` feature  to a __datetime object__ as it allows us to __extract the time features__ such as the __month__ and __year__ of the request date.

We will also be dropping the `request_date` feature after extracting the time features as we __do not need the level of granularity of having individual dates__. One hot encoding the feature would result in a __sparse matrix__ as well.

In [24]:
# converting to datetime format
train['request_date'] = pd.to_datetime(train['request_date'], format='%d-%b-%y')

In [25]:
# creating new features for the year and month that the loan was requested and dropping original feature
train = cleaning.create_time_vars(train, time_var='request_date', year=True, month=True,
                                  day=False, season=False, drop=True)

Datetime variable has been dropped.


Additionally our numerical features are expected to be postive, therefore we need to chekc if there are any negative values which may signal some error during the data collection stage.

In [27]:
# checking for negative values
negative_values = cleaning.check_negative(train)
negative_values

There are no numeric variables with negative values.


,NegativeValues
term,False
employee_count,False
business_type,False
loan_amount,False
insured_amount,False
default_status,False


### Handling Missing Data <a id='missing_data'></a>

Now that we have ensured the data types are correct and have sanity checked our data, we can move on to handling missing data.

In [28]:
# displays features with missing data and proportion of data that is missing
cleaning.variable_missing_percentage(train)

The dataframe has 13 variables.
There are a total of 1 variables with missing values and their missing percentages are as follows:

industry    0.04


There is only one feature with missing values. As the __percentage missing is small__, we will just __remove the data points__.

In [29]:
train = train.dropna(how='any')
cleaning.variable_missing_percentage(train)

The dataframe has 13 variables.
The dataframe has no missing values in any column.


Now that we have sanitised our data sufficiently, we can move on to analysing our features individually.

[Return to Table of Contents](#table_contents)